In [1]:
"""IMPORT PACKAGES"""
import numpy as np #array magic from numpy 
from scipy.stats import lognorm #import the log-normal distribution for choosing sphere sizes from it 
import copy #to make deep copies which do not change the original array they are copied from
import time # to time stuff 
from stack_tools import *
from move_tools import *
import mayavi
from mayavi import mlab
%gui qt
from IPython.display import clear_output
import itertools

In [2]:
%load_ext autoreload
%autoreload 2 

In [3]:
def sequential_addition(particles,mgrid,dims,delta,grid_size,debug): 
    """Take in an array of spherical particles [r,x,y,z,t] where r is radius, x,y,z are position coordinates-- z needs to be
    higher tha any point in the mgrid, mgrid is a grid containing the particles, save==True generates folder named folder"""
    for i,p in enumerate(particles):
        clear_output(wait=True)
        print('iteration ' +str(i))
        p,debug = rearrange_sa(p,grid_size,dims,delta,mgrid)
        
def collective_rearrangement(particles,mgrid,dims,delta,grid_size):
    """take in a spatial mgrid filled with potentially overlapping particles [r,x,y,z,t] (radius,position,tracer ind)
    and rearrange them until they no longer overlap"""
    for i,p in enumerate(particles):
        clear_output(wait=True)
        print('iteration ' +str(i))
        s = rearrange_cr(p,grid_size,dims,delta,mgrid)

### Example A: Generate 500 uniform spheres with uniformly distributed radius within a 5x5x5 grid, then sequentially add them into a dense packing

I think find Odir is broken when N_cons >=2

In [68]:
dims = [10,10,10]
mdims = list(np.array(dims)+np.array([2,2,0]))
NX,NY,NZ = dims
delta = 0.1
radii_array = np.ones(5000,dtype=float)
#radii_array = np.random.uniform(size=5000)*0.8+0.2
#radii_array = np.random.lognormal(mean=0.5,sigma=0.2,size=5000)
#delta = np.amin(radii_array)/4
grid_size = np.amax(radii_array)*2+delta
spheres = initialize_spheres(radii_array,dims,grid_size,delta,seq_add=True)
#for s in spheres:
#    s.position = np.array([NX/2,NY/2,NZ/2],dtype=float)*grid_size+np.random.random(size=3)*delta
mgrid = initialize_grid(mdims)

In [69]:
debug = []
for i,p in enumerate(spheres):
    clear_output(wait=True)
    print('iteration ' +str(i))
    
    NX,NY,NZ = dims
    s = copy.deepcopy(p)
    s.state=0    
    
    while True: #this loop just passes the particle through G,D,W,O,OO,R as necessary
        print(s.position,s.radius,s.state)
        
        cons = c_sort(s,grid_size,delta,mgrid)
        if s.state==0:
            s = G_sa(s,cons,grid_size,dims,delta,mgrid)
            continue 
        elif s.state==1:
            s = D(s,grid_size,delta,mgrid)
            continue
        elif s.state==2:
            s = W(s,grid_size,dims)
            continue
        elif s.state==3:
            s = O(s,cons,grid_size,dims,delta,mgrid)
            continue 
        elif s.state==4: 
            s = OO(s,cons,grid_size,dims,delta,mgrid)
            continue
        elif s.state==5:
            if indices(s,grid_size)[-1] < NZ: 
                s = R(s,grid_size,dims,mgrid)
            else: 
                print('Particle past max height')
                s = None 
            break    
    


iteration 280
(array([6.52126196e-01, 8.84789856e-03, 2.31000000e+01]), 1.0, 0)
(array([6.52126196e-01, 8.84789856e-03, 2.31000000e+01]), 1.0, 1)
0 before D
0 after D
(array([0.6521262, 0.0088479, 5.379    ]), 1.0, 3)
0 before O
[5.338698177762397]: zees
[-0.04897815402812743]: thes
[array([-0.45938481, -0.88823735,  0.        ])]: axes
[array([0.6521262, 0.0088479, 5.379    ])] p position
[array([-0.03846166,  0.36601096,  3.51319873])]: contacts
attempting drop out of overlap generating O


AttributeError: 'numpy.ndarray' object has no attribute 'position'

In [66]:
s.position

array([-43.03943838, -17.16561086,   1.0014588 ])

In [67]:
view(mgrid)

In [ ]:
s.tag [q.tag for q in itr(mgrid)]

In [ ]:
view_condition(debug,lambda x: x.tag == s.tag)
view(debug)

In [ ]:
p = None
q = None

In [ ]:
p.position

In [ ]:
import numpy as np
import math

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

v = [3, 5, 0]
axis = [4, 4, 1]
theta = 1.2 

print(np.dot(rotation_matrix(axis,theta), v)) 
# [ 2.74911638  4.77180932  1.91629719]

In [ ]:
#need to test the rotator 
x = 2.1*5/2
dims = (5,5,5)
delta = 0.5
grid_size = 2.1
mgrid = initialize_grid((7,7,5))

p = sphere(1.0,1,dims,grid_size,delta)
p.position = np.array([x,x,x],dtype=float)

q = sphere(1.0,2,dims,grid_size,delta)
q.position = np.array([x-delta/10,x,x-1.85],dtype=float)

q = R(q,grid_size,dims,mgrid)
p  = R(p,grid_size,dims,mgrid)

#m = sphere(1.0,3,dims,grid_size,delta)
#m.position = 

In [ ]:
[p.position for p in itr(mgrid)]

In [ ]:
contacts(p,grid_size,delta,mgrid)

In [ ]:
cons = contacts(p,grid_size,delta,mgrid)

In [ ]:
dthe,A,P = find_Odir(p,cons,grid_size,(5,5,5),delta,mgrid)

In [ ]:
parts = []
for i in range(200):
    p.position = rot(dthe,A,P,p)
    parts.append(p.position)


In [ ]:
for p in parts:
    R(p,grid_size,dims,mgrid)

In [ ]:
view(mgrid)

In [ ]:
[p.position for p in parts]

In [ ]:
h = sphere(1.0,3,dims,grid_size,delta)
h.position = np.array([5.70,5.25,5.061])
h = R(h,grid_size,dims,mgrid)

In [ ]:
view(mgrid)

In [ ]:
q.position

In [ ]:
def find_Odir(p,cons,grid_size,dims,delta,mgrid):
    NX,NY,NZ = dims 
    zees = [] #z value after rotation
    thes = [] #rotation angle increments to sweep through delta
    axes = [] #rotation axes
    pivs = [] #pivot points 
    x,y,z = p.position #unpack particle position
    for c in cons:
        #print(p.position)
        #print(c.position)
        c1,c2,c3 = P = c.position #pivot point
        dthe = 0.99*delta/np.linalg.norm(p.position-c.position) #unsigned rotation increment 
        A = np.array([y-c2,c1-x,0]) #rotation axis 
        if np.linalg.norm(A)>0:
            A = A/np.linalg.norm(A)
        for sign in [-1,1]:
            o = copy.deepcopy(p) #a copy of p to test the rotation 
            dthe = sign*dthe #signed rotation increment
            print('before: '+str(o.position))
            o.position = rot(dthe,A,P,o) #perform the test rotation
            print('med' + str(o.position))
            if not within_grid(o,grid_size,dims): #if test rotation left boundary, wrap it back
                o = W(o,grid_size,dims)
            if np.isnan(o.position[0]):
                return (dthe,A,P,o)
        
            print('after: '+str(o.position))

            Z = o.position[-1] #what is the new value of Z induced by the test rotation?
            Z0 = p.position[-1]
            N_ov = len(overlaps(o,grid_size,delta,mgrid)) #how many overlaps were generated by the test rotation?
            if  Z<Z0 and N_ov==0:#it o is lower than p and has no overlaps
                zees.append(Z) #add potential movement parameters 
                thes.append(dthe)
                axes.append(A)
                pivs.append(P)
    #now among the possibilities find the optimal set of parameters 
    if len(zees)>0: #find largest decrease in z 
        min_z, idx = min((val, idx) for (idx, val) in enumerate(zees)) #smallest z coordinate after rotation
        A = axes[idx] #largest decrease in z through rot for particle p induced by these parameters
        P = pivs[idx]
        dthe = thes[idx]
        return (dthe,A,P) #return the tuple of optimal rotation parameters
    else:
        return None #return statement that O fails 

In [ ]:
rot(,A,P,o) #perform the test rotation


### Example B: Generate 300 uniform size spheres within a 5x5x5 grid, then collectively rearrange them into a dense packing

In [ ]:
s.tag

In [ ]:
NX,NY,NZ = dims = (5,5,5)
mgrid = initialize_grid(dims = (NX+2,NY+2,NZ))
delta = 0.05
radius = 0.5  
grid_size = 2*radius+delta #2*max_radius + delta
N = 300
radii_array = np.ones(N)*radius
spheres = initialize_spheres(radii_array,dims,grid_size,delta)
collective_rearrangement(spheres,mgrid,dims,delta,grid_size)
view(mgrid)

In [ ]:
view(mgrid)

# transport model development

In [ ]:
#just make a bed for testing the transport model 
NX,NY,NZ = dims = [20,20,4]
mgrid0 = initialize_grid((NX+2,NY+2,NZ))
delta = 0.05
radius = 0.5  
grid_size = 2*radius+delta #2*max_radius + delta
N = 2000
radii_array = np.ones(N)*radius
spheres = initialize_spheres(radii_array,dims,grid_size,delta)
collective_rearrangement(spheres,mgrid0,dims,delta,grid_size)

In [ ]:
#now make z bigger 
NZ = 2*NZ
dims[-1]=NZ
mgrid = extend_mgrid(mgrid0,NZ)
mgrid0 = copy.deepcopy(mgrid)
view(mgrid)

In [ ]:
mgrid = copy.deepcopy(mgrid0)

## verbose globally updating transport model

In [ ]:
#reset
mgrid = copy.deepcopy(mgrid0)
#initialize the model only once
clusters = determine_clusters(mgrid,grid_size,delta)  
#generate transport array
transport = initialize_grid(dims[:-1])

In [ ]:
view_clusters(mgrid,clusters)

In [ ]:
i=0
transport_rate=[]
travel_times = []
entrainment_rate = []
deposition_rate = []
N=len(itr(mgrid))
print(str(len(itr(mgrid)))+' particles initially in system.')
while True:
    i+=1
    
    clear_output(wait=True)
    #determine the clusters to be entrained
    print('-----------------------------------------------------------------------')
    print('ITERATION '+str(i))
    print('-----------------------------------------------------------------------')
    entrained_clusters = determine_entrained(clusters)
    #print(str(len(entrained_clusters))+' clusters to be entrained')
    #print(str(len([cp for ec in entrained_clusters for cp in ec.particles]))+ ' particles to be entrained')
    #print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')
    #print(str(len(fullitr(transport)))+ ' particles in transport')
    
    #actually entrain the clusters into the transport array 
    #n1 = len(itr(mgrid))
    #m1 = len(fullitr(transport))
    #l1 = n_unstable(mgrid,delta,grid_size)
    #####################
    clusters = entrain_into_transport(transport,clusters, mgrid,grid_size,delta,*entrained_clusters) #modifies transport in place
    #####################
    #n2 = len(itr(mgrid))
    #m2 = len(fullitr(transport))
    #entrainment_rate.append(m2-m1)
    #l2 = n_unstable(mgrid,delta,grid_size)
    #if not -(n2-n1)==m2-m1:
    #    print('number conservation fail in entrain_into_transport')
    #    break
    #if l1==0 and l2!=0:
    #    print('unstable particles generated from entrain_into_transport')
    #    break

    #print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')

    
    #reconstruct clusters array following modification of bed surface
    #print(len(clusters), ' clusters initially')
    #print(len([p for cl in clusters for p in cl.particles]), ' particles in clusters initially')
    ###############################
    #clusters = determine_clusters(mgrid,grid_size,delta)
    ####################################
    #print(len(clusters), ' clusters after redeterminiation of clusters')
    #print(len([p for cl in clusters for p in cl.particles]), ' particles in clusters after redetermination of clusters')
    #print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')

    
    #transport the active particles 
    #m1 = len(fullitr(transport))
    ###########################
    transport = translate_by_timestep(transport,grid_size,delta,dims)
    ############################
    #m2 = len(fullitr(transport))
    #print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')
    
    
    #determine particles to be disentrained
    particles_to_deposit = determine_deposited(transport)
    #print(len(particles_to_deposit), 'particles slated for deposition')
    #print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')
    
    
    #disentrain the particles back onto the bed 
    #n1 = len(fullitr(transport))
    #m1 = len(itr(mgrid))
    #l1 = n_unstable(mgrid,delta,grid_size)
    transport,travel_iterations = deposit_onto_bed(transport,mgrid,grid_size,delta,dims,*particles_to_deposit)
    travel_times.extend(travel_iterations)
    #l2 = n_unstable(mgrid,delta,grid_size)
    #n2 = len(fullitr(transport))
    #deposition_rate.append(n1-n2)
    #m2 = len(itr(mgrid))
    #if not -(n2-n1)==m2-m1:
    #    print('number conservation fail in deposit_onto_bed')
    #    break
    #if l1==0 and l2!=0:
    #    print('deposit_onto_bed generated an instability')
    #    break
    #print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')
    
    
    #redetermine clusters array following modification of bed surface

    #print(len(clusters), ' clusters initially')
    #print(len([p for cl in clusters for p in cl.particles]), ' particles in clusters initially')
    clusters = determine_clusters(mgrid,grid_size,delta)
    #clus3inclus2(clusters)
    #print(len(clusters), ' clusters after redeterminiation of clusters')
    #print(len([p for cl in clusters for p in cl.particles]), ' particles in clusters after redetermination of clusters')
    #print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')
    #print(str(len(itr(mgrid))+len(fullitr(transport)))+' particles total')
    #print(str(len(fullitr(transport)))+ ' particles in transport')
    transport_rate.append(len(fullitr(transport)))

    if n_unstable(mgrid,delta,grid_size)!=0:
        print('unstable break')
        break
    #if len(itr(mgrid))+len(fullitr(transport))!=N:
    #    print('particle conservation break')
    #    break
    


In [ ]:
?T

In [ ]:
np.save('travel_times.npy',travel_times)
np.save('transport_rate.npy',transport_rate)
np.save('entrainment_rate.npy',entrainment_rate)
np.save('deposition_rate.npy',deposition_rate)

In [ ]:
clus3inclus2(clusters)

In [ ]:
view_clusters(mgrid,clusters)

question: 
can a 3 cluster contain a 2 cluster?
answer: 
yes. 
idea:
incorporate equality between particles... p1==p2 if their tags are the same


In [ ]:
def clus3inclus2(clusters):
    clusters2 = [clus for clus in clusters if len(clus.particles)==2]
    clusters3 = [clus for clus in clusters if len(clus.particles)==3]
    tags2 = [p.tag for cl in clusters2 for p in cl.particles]
    tags3 = [p.tag for cl in clusters3 for p in cl.particles]
    for t2 in tags2:
        for t3 in tags3:
            for t in t2.split('_'):
                for w in t3.split('_'):
                    if t==w:
                        print('a 2 in a 3')
            

In [ ]:
clusters[0].__dict__

In [ ]:
def degenerate_clusters(clusters):
    cparticles = [clu.particles for clu in clusters]
    parts = [p for clup in cparticles for p in clup]
    parts2=[]
    for clus in cparticles:
        for dlus in cparticles:
            for c in clus:
                for d in dlus:
                    if c.tag==d.tag:
                        parts2.append(c)
    if len(parts)!=len(parts2):
        print(str(len(parts2)-len(parts))+' degenerate clusters in system.')             

## local updating transport model

In [ ]:
#here is the model
for i in range(100):
    print('iteration '+str(i))
    print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')
    
    entrained_clusters = determine_entrained(clusters)
    
    print(str(len(entrained_clusters))+ ' entrained clusters')
    print(' len clusters array before entrainment: ' + str(len(clusters))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    
    clusters = entrain_clusters(clusters,mgrid,grid_size,delta,*entrained_clusters)
    
    print(' len clusters array after entrainment: ' + str(len(clusters))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    print(' number of particles in transport array before entrainment: ' + str(len(fullitr(transport)))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    
    transport = entrain_into_transport(transport,mgrid,grid_size,delta,*entrained_clusters)
    
    print(' number of particles in transport array after entrainment: ' + str(len(fullitr(transport)))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    
    transport = translate_by_timestep(transport,grid_size,delta,dims)
    
    print(' number of particles in transport array after displacement: ' + str(len(fullitr(transport)))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    
    deposited_clusters = determine_deposited(transport)
    
    print(str(len(deposited_clusters))+ ' particled slated for deposition from transport array')
    print(str(len(itr(mgrid)))+' particles in mgrid before deposition.'+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    
    transport,clusters = deposit_onto_bed(transport,clusters,mgrid,grid_size,delta,dims,*deposited_clusters)     
    
    print(str(len(itr(mgrid)))+' particles in mgrid after deposition.'+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable') 
    print('--------------------------------------------------------------')    

In [ ]:
for i in range(100):
    print('iteration '+str(i))
    print(str(n_unstable(mgrid,delta,grid_size))+ ' particles unstable total')
    #print(len(itr(mgrid))+len(fullitr(transport)),' total particles at iteration ', i)
    entrained_clusters = determine_entrained(clusters)[:1]
    clusters = determine_clusters(mgrid,grid_size,delta)
    print(str(len(entrained_clusters))+ ' entrained clusters')
    print(' len clusters array before entrainment: ' + str(len(clusters))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    mgrid_bug = copy.deepcopy(mgrid)
    clusters_bug = copy.deepcopy(clusters)
    entrained_clusters_bug = copy.deepcopy(entrained_clusters)
    
    clusters = entrain_clusters(clusters,mgrid,grid_size,delta,*entrained_clusters)
    
    if n_unstable(mgrid,delta,grid_size)!=0:
        print('broken')
        break
        
    print(' len clusters arrmay after entrainment: ' + str(len(clusters))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    print(' number of particles in transport array before entrainment: ' + str(len(fullitr(transport)))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    transport = entrain_into_transport(transport,mgrid,grid_size,delta,*entrained_clusters)
    print(' number of particles in transport array after entrainment: ' + str(len(fullitr(transport)))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    transport = translate_by_timestep(transport,grid_size,delta,dims)
    print(' number of particles in transport array after displacement: ' + str(len(fullitr(transport)))+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    deposited_clusters = determine_deposited(transport)
    print(str(len(deposited_clusters))+ ' particled slated for deposition from transport array')
    print(str(len(itr(mgrid)))+' particles in mgrid before deposition.'+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable')
    contacts_above_in_clusters(clusters,mgrid,flag='before deposition')

    transport,clusters = deposit_onto_bed(transport,clusters,mgrid,grid_size,delta,dims,*deposited_clusters)
    contacts_above_in_clusters(clusters,mgrid,'after depsosition')
    print(str(len(itr(mgrid)))+' particles in mgrid after deposition.'+ '; '+ str(n_unstable(mgrid,delta,grid_size))+ ' unstable') 
    print('--------------------------------------------------------------')

In [ ]:
mgrid =     copy.deepcopy(mgrid_bug) 
clusters =     copy.deepcopy(clusters_bug)
entrained_clusters = copy.deepcopy(entrained_clusters_bug)
transport = translate_by_timestep(transport,grid_size,delta,dims)


In [ ]:
def filter_invalid_clusters(clusters_to_check,clusters,mgrid,grid_size,delta):
    """check whether a list of clusters are all valid in terms of contingency"""
    new_clusters = []
    for clu in clusters_to_check: 
        #find the index of clu in clusters
        j = clusters.index(clu) #this is the index of clu in clusters
        cluclusters = []
        cluclusters = update_clusters(clu.particles,cluclusters,mgrid,grid_size,delta)
        #does clu remain in cluclusters? if so, ok. if not, must delete clu from clusters
        if clu not in cluclusters:
            del clusters[j] #remove clu from clusters, because deposition removed it from cluster status 

        clusters.extend([qlu for qlu in cluclusters if not qlu==clu]) #either way, add any new clusters 
    return clusters          

In [ ]:
def filter_clusters_from_deposition(settled_particles,clusters,mgrid,grid_size,delta):
    """depo_cons are the contacts of recently deposited particles. If they are also within clusters,
    these clusters need to be reconsidered"""
    clusters_to_reconsider=[]
    particles_to_reconsider_for_clustering = []
    for clus in clusters: #take a cluster already within clusters. ask if it is affected by deposition
        #if settled_particles landed on a cluster, need to reexamine all settled particles AND the settled particle
        flag=True# each cluster in need of examination will be added to the examination list only once
        for c in clus.particles: #for each particle in the cluster
            for s in settled_particles: #for each particle which has settled
                if s.radius+c.radius-delta<distance(c,s)<s.radius+c.radius+delta: #if cluster particle in contact with settled particle
                #if s in contact with cluster 
                    if flag: 
                    #if this is the first such s to contact this cluster 
                        particles_to_reconsider_for_clustering.extend(clus.particles) #then reconsider its particles
                        clusters_to_reconsider.append(clus) # and reconsider its status as a cluster 
                        flag=False #but only once per cluster
                    particles_to_reconsider_for_clustering.append(s) #and also must consider recently deposited s 
    
    #now need to take clusters list and filter it over clusters_to_reconsider. method is filter_invalid_clusters 
    #need a function that checks if a cluster is valid 
    
    
    
    
    #finally need to take particles_to_reconsider_for_clustering list and use update_clusters on it
    clusters = update_clusters(particles_to_reconsider_for_clustering,clusters,mgrid,grid_size,delta)
    return clusters 